# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.  
这一节的内容实际就是RAG。使用VectorstoreIndexCreator几行代码就实现了基本的RAG功能。

In [1]:
from langchain.chains import RetrievalQA
#from langchain.chat_models import ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI

1. 使用ollama下载deepseek-r1:32b  
2. 后台启动ollama服务： nohup ollama serve  >/dev/null 2>&1 &  
3. 我用的是3090, 共24G显存，占用了21G。

In [10]:
llm = ChatOpenAI(
    model_name='deepseek-r1:32b',
    openai_api_base="http://127.0.0.1:11434/v1",
    openai_api_key="EMPTY",
    streaming=False
)

In [2]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

这里创建的VectorstoreIndexCreator对象index就是向量数据库。

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings

# 初始化 Hugging Face 嵌入模型
embeddings = HuggingFaceEmbeddings()

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

/tmp/ipykernel_240605/2339249052.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [7]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

向量数据库index，传入query和llm，就是一个最基本的RAG。

In [11]:
response = index.query(query, 
                       llm = llm)

In [12]:
display(Markdown(response))

<think>
Alright, so I just received this query asking me to list all their shirts with sun protection in a table using markdown and summarize each one. Let's see what I need to do here.

First, the user has provided four context entries, each about a different shirt. Each entry includes details like product ID, name, description, size and fit, fabric and care, additional features, etc. My task is to extract relevant information from these contexts to create a table.

I think the best approach is to identify common categories that can be used across all shirts. Looking at each context, I see they mention sun protection, UPF rating, fabric composition, style (sleeve), fit, and any special features like moisture-wicking or ventilation.

Now, for each shirt, I'll need to extract this info. Let me go through them one by one.

1. The first shirt is "Women's Tropical Tee, Sleeveless" with ID 679. From the description, it has UPF 50+ and blocks 98% UV rays. Fabric is Shell: 71% nylon and 29% polyester; Cape lining is 100% polyester. It's sleeveless, slightly fitted, and has features like smoother buttons, wrinkle resistance, low-profile pockets, etc.

2. The second shirt is "Sun Shield Shirt by" with ID 255. UPF 50+, same as the first. Fabric: 78% nylon, 22% Lycra Xtra Life. It's also sleeveless, slightly fitted, moisture-wicking, and has abrasion resistance.

3. The third shirt is "Men's Plaid Tropic Shirt, Short-Sleeve" with ID 374. UPF 50+ again. Fabric is 52% polyester, 48% nylon. Short sleeves, traditional fit. Features include front/back cape venting, two pockets, moisture-wicking.

4. The fourth shirt is "Tropical Breeze Shirt" with ID 619. Also UPF 50+. Fabric: Shell 71% nylon, 29% polyester; lining 100% polyester; mesh inserts. It's long-sleeve, traditional fit. Features include wrinkle resistance and moisture-wicking.

Okay, so compiling this info into a table makes sense. The columns will be Product ID, Name, Sun Protection (UPF), Fabric, Sleeve Style, Fit, and Special Features.

Now, for each row:

- For the Women's Tropical Tee: product 679, sleeveless, slightly fitted. Fabric details as above. Features include smoother buttons, wrinkle resistance, etc. I can mention some key features like pocket tabs and eyewear loop.

- Sun Shield Shirt: product 255, same UPF, different fabric blend, sleeveless, slightly fitted. Additional features are moisture-wicking, fits over swimsuit, abrasion-resistant.

- Men's Plaid Tropic: product 374, short sleeves, traditional fit, polyester/nylon. Features include pockets and venting.

- Tropical Breeze: product 619, long-sleeve, traditional fit, same fabric as one of the other shirts. Features are wrinkle-resistant and moisture-wicking.

I think this covers all four shirts. Now, just to make sure I haven't missed any key details. Each shirt has UPF 50+ and uses synthetic fabrics for sun protection. They differ in sleeve style (sleeveless, short, long) and fit. Special features vary but include things like pockets, venting, moisture-wicking, etc.

So, structuring this into a table where each column holds the relevant info should answer the user's query effectively.
</think>

Here is a table listing all the shirts with sun protection, along with a summary of their key features:

```markdown
| Product ID | Name                          | Sun Protection (UPF) | Fabric Composition                               | Sleeve Style       | Fit             | Special Features                                                                 |
|-----------:|-------------------------------|----------------------|-------------------------------------------------|-------------------|----------------|----------------------------------------------------------------------------------|
| 679        | Women's Tropical Tee         | UPF 50+             | Shell: 71% nylon, 29% polyester; Lining: 100% polyester | Sleeveless          | Slightly Fitted | SunSmart™ technology, smoother buttons, wrinkle resistant, front/back Cape venting, tool tabs, eyewear loop.      |
| 255        | Sun Shield Shirt             | UPF 50+             | Shell: 78% nylon, 22% Lycra Xtra Life fiber       | Sleeveless          | Slightly Fitted | Moisture-wicking, fits over swimsuit, abrasion-resistant.                                            |
| 374        | Men's Plaid Tropic Shirt     | UPF 50+             | Shell: 52% polyester, 48% nylon                   | Short sleeve        | Traditional Fit | Front/back cape venting, two front bellows pockets, moisture-wicking, imported.                       |
| 619        | Tropical Breeze Shirt        | UPF 50+             | Shell: 71% nylon, 29% polyester; Lining: 100% polyester, mesh inserts | Long sleeve         | Traditional Fit | Wrinkle-resistant, moisture-wicking, breathable.                                                      |
```

### Summary of Each Shirt:
1. **Women's Tropical Tee** (Product ID 679):
   - A sleeveless button-up shirt with UPF 50+ sun protection.
   - Features a slightly fitted design, wrinkle resistance, and additional functional details like tool tabs and an eyewear loop.

2. **Sun Shield Shirt** (Product ID 255):
   - Another sleeveless option with UPF 50+ protection.
   - Designed to be moisture-wicking and durable (abrasion-resistant), ideal for active use or layering over swimsuits.

3. **Men's Plaid Tropic Shirt** (Product ID 374):
   - A short-sleeve shirt with UPF 50+ protection in a traditional fit.
   - Features front and back cape venting, two front bellows pockets, and moisture-wicking properties.

4. **Tropical Breeze Shirt** (Product ID 619):
   - A long-sleeve shirt with UPF 50+ protection, designed for breathability and comfort in warm weather.
   - Features wrinkle-resistant fabric and moisture-wicking technology.

## Step By Step

上面我们用几行代码就实现了最基本的RAG功能，下面我们来详细看看内部发生了什么。

In [13]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [14]:
docs = loader.load()

In [15]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [18]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [19]:
print(len(embed))

768


In [20]:
print(embed[:5])

[0.033366858959198, -0.004217786714434624, 0.005565309897065163, -0.0070280712097883224, 0.010666914284229279]


In [21]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [22]:
query = "Please suggest a shirt with sunblocking"

In [23]:
docs = db.similarity_search(query)

In [24]:
len(docs)

4

In [25]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.')

In [26]:
retriever = db.as_retriever()

In [27]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [28]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


/tmp/ipykernel_240605/2552755760.py:1: LangChainDeprecationWarning: The method `BaseChatModel.call_as_llm` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.call_as_llm(f"{qdocs} Question: Please list all your \


In [29]:
display(Markdown(response))

<think>
Alright, the user has provided several shirt descriptions, each with details about their features, composition, care instructions, etc. They want me to list all the shirts with sun protection in a table using Markdown and then summarize each one.

First, I should go through each of the four shirt entries. Let me read them carefully.

1. **Sun Shield Shirt by**: It mentions high-performance fabric with UPF 50+ from The Skin Cancer Foundation. It's made of 78% nylon and 22% Lycra Xtra Life, handwash required. Additional features include moisture-wicking, fits over swimsuits, abrasion-resistant, imported.

2. **Women's Tropical Tee, Sleeveless**: This one has a five-star sleeveless design with a built-in SunSmart™ UPF 50+. Made of 71% nylon and 29% polyester shell, plus a polyester lining. Machine washable. Features include smoother buttons, wrinkle-resistant, pockets, venting, tool tabs, eyewear loop, imported.

3. **Girls' Beachside Breeze Shirt, Half-Sleeve**: Designed for beach or watersports with UPF 50+. Made of 80% nylon and 20% Lycra Xtra Life, machine wash and line dry. Features: rash guard, snag- and fade-resistant, seawater-resistant, durable seams, imported.

4. **Sunrise Tee**: Women's button down with UPF 50+ from SunSmart™. Made of lightweight synthetic (71% nylon and 29% polyester shell) with a polyester lining. Features: moisture-wicking, wrinkle-free, smoother buttons, two front pockets, tool tabs, eyewear loop. Machine washable.

Now, for the table. I'll need columns: Name, Description, Fabric & Care, UPF Rating, Key Features, Importance of Protection.

I should ensure each shirt is accurately represented, highlighting the sun protection aspects and other key features.

Looking at UPF ratings: all have UPF 50+ except the Girls' one which also mentions blocking 98% of UV rays. I'll note that in the summary.

The summaries need to be concise but comprehensive. For each shirt, I'll mention target audience (women or girls), purpose, fabric composition, sun protection details, key benefits like moisture-wicking, wrinkle resistance, any unique features like side shaping or tool tabs, and washing instructions.

I should also note that all are imported except the Girls' which might not be? Wait, let me check: Sun Shield is imported, Women's Tropical is imported, Girls' is imported too, and Sunrise is imported. So all are imported.

I think that's all. Now I'll structure this into a table, ensuring each point is covered, then provide the summary for each shirt.
</think>

Here is a table summarizing the shirts with sun protection, along with key details:

| **Name**                        | **Description**                                                                                       | **Fabric & Care**                                      | **UPF Rating**          | **Key Features**                                                                                         |
|---------------------------------|-------------------------------------------------------------------------------------------------------|------------------------------------------------------|------------------------|----------------------------------------------------------------------------------------------------------|
| Sun Shield Shirt by             | High-performance sun shirt blocking harmful UV rays.                                                    | 78% nylon, 22% Lycra Xtra Life fiber. Handwash, line dry.      | UPF 50+              | Moisture-wicking, fits over swimsuits, abrasion-resistant, imported.                                     |
| Women's Tropical Tee, Sleeveless| Five-star sleeveless button-up with SunSmart™ protection.                                               | Shell: 71% nylon, 29% polyester; lining: 100% polyester. Machine wash and dry.          | UPF 50+              | Smoother buttons, wrinkle-resistant, low-profile pockets, front and back venting, imported.               |
| Girls' Beachside Breeze Shirt   | rash guard with extra coverage for watersports or beach days.                                           | 80% nylon, 20% Lycra Xtra Life. Machine wash, line dry.      | UPF 50+              | Durable, seawater-resistant, snag- and fade-resistant, imported.                                            |
| Sunrise Tee                     | Lightweight UV-protective button-down shirt for hot days.                                               | Shell: 71% nylon, 29% polyester; lining: 100% polyester. Machine wash and dry.          | UPF 50+              | Wrinkle-free, moisture-wicking, smoother buttons, front and back venting, imported.                       |

---

### **Summary of Each Shirt**

1. **Sun Shield Shirt by**  
   - A high-performance sun shirt designed to block harmful UV rays with UPF 50+.  
   - Made from durable 78% nylon and 22% Lycra Xtra Life fiber, it wicks moisture and fits comfortably over swimsuits.  
   - Abrasion-resistant and imported.

2. **Women's Tropical Tee, Sleeveless**  
   - A stylish sleeveless button-up shirt with SunSmart™ UPF 50+ protection.  
   - Constructed from a blend of nylon and polyester for wrinkle resistance and comfort.  
   - Features include smoother buttons, low-profile pockets, and front/back venting for a flattering fit. Imported.

3. **Girls' Beachside Breeze Shirt, Half-Sleeve**  
   - A versatile rash guard designed for beach days or watersports with UPF 50+ protection.  
   - Made from 80% nylon and 20% Lycra Xtra Life fiber, it is durable, seawater-resistant, and fade-resistant.  
   - Designed to hold its shape and color while offering full sun protection.

4. **Sunrise Tee**  
   - A lightweight, UV-protective button-down shirt for hot days with UPF 50+.  
   - Made from a blend of nylon and polyester for a smooth, wrinkle-free finish.  
   - Features moisture-wicking properties, smoother buttons, and front/back venting for breathability. Imported.

Each shirt offers excellent sun protection with additional features tailored to specific needs, such as durability, comfort, or style.

In [30]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [31]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [32]:
response = qa_stuff.run(query)

/tmp/ipykernel_240605/3485144879.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_stuff.run(query)




> Entering new RetrievalQA chain...

> Finished chain.


In [33]:
display(Markdown(response))

<think>
Okay, so I need to help the user by listing all their shirts that have sun protection in a table using markdown and then summarizing each one. Let me see how to approach this.

First, looking at the context provided, there are four entries: 679, 709, 255, and 619. Each of these has information about their names, descriptions, sizes, fabrics, care instructions, additional features, and sun protection details. 

I'll start by compiling each shirt's details for the table. The table should include columns like Product Code, Name, Key Features, Sun Protection, Fabric & Care, and Size & Fit.

For product 679: Women's Tropical Tee, Sleeveless. It has UPF 50+ protection from its shell fabric of nylon and polyester with a polyester lining. It's machine washable. The size is Slightly Fitted, falls at hip. Additional features include smoother buttons, wrinkle resistance, venting, pockets, etc.

Product 709: Sunrise Tee. Also UPF 50+, same fabric as the first one. This shirt is designed to be lightweight and moisture-wicking, beat the heat, good for fishing or travel. It's slightly fitted, falls at hip.

Product 255: Sun Shield Shirt by... Wait, the name says 'Sun Shield shirt by' but it's cut off. Probably a brand name missing. The fabric here is different; 78% nylon and 22% Lycra Xtra Life. It needs hand washing. Features include moisture-wicking, can be worn over swimsuits, abrasion resistant.

Product 619: Tropical Breeze Shirt (men’s). Long-sleeve for men, UPF 50+. Traditional fit, relaxed. It's breathable and good for outdoor activities like fishing or travel. The fabric is similar to the first two, with mesh inserts for breathability. Machine washable.

Now, putting all this into a markdown table seems straightforward. I'll structure it with each product as a row, listing its details across each column.

After the table, summarizing each shirt would involve highlighting their unique features and uses. For example, the Women's Tropical Tee is sleeveless and good for general use, while the Sunrise Tee is light and great for travel. The Sun Shield Shirt is more sporty with Lycra, and the Tropical Breeze is specifically for men with a relaxed fit.

I should make sure that each summary is concise but informative, capturing what makes each shirt special beyond just having sun protection.
</think>

Here is a detailed presentation of the shirts with sun protection, including a markdown table and summaries:

| Product Code | Name                       | Key Features                                      | Sun Protection       | Fabric & Care                             | Size & Fit    |
|--------------|----------------------------|--------------------------------------------------|----------------------|-------------------------------------------|---------------|
| 679          | Women's Tropical Tee, Sleeveless | Slightly fitted, UPF 50+, moisture-wicking      | UPF 50+             | Shell: nylon/polyester, lining: polyester | Slightly Fitted |
| 709          | Sunrise Tee                | Lightweight, wrinkle-free, moisture-wicking     | UPF 50+             | Same as above                             | Slightly Fitted |
| 255          | Sun Shield Shirt           | Moisture-wicking, fits over swimsuit            | SPF 50+, UPF 50+    | Nylon/Lycra Xtra Life               | Slightly Fitted     |
| 619          | Tropical Breeze Shirt      | Long-sleeve, breathability, mesh inserts        | UPF 50+             | nylon/polyester, mesh inserts           | Traditional Fit   |

### Summaries:

1. **Women's Tropical Tee, Sleeveless (Product 679)**: Ideal for general outdoor use, this sleeveless shirt offers UPF 50+ protection with a flattering fit. It's designed to keep you cool and protected.

2. **Sunrise Tee (Product 709)**: A lightweight option perfect for travel or sports like fishing. It combines sun protection with moisture-wicking properties for comfort in hot weather.

3. **Sun Shield Shirt (Product 255)**: Designed for active wear, this shirt can be worn over swimsuits and offers durable SPF 50+ protection. Its combination of nylon and Lycra ensures performance and comfort.

4. **Tropical Breeze Shirt (Product 619)**: Tailored for men with a relaxed fit, this long-sleeve shirt provides UPF 50+ protection and is ideal for outdoor activities requiring breathability and ease of movement.

Each shirt is crafted to offer superior sun protection while addressing specific needs, whether it's comfort, style, or performance.

In [34]:
response = index.query(query, llm=llm)

In [35]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])